# Load data via the API


Before running this notebook, initialize the database:

```
docker-compose exec web flask db init
```

In [1]:
import requests
import json
import os

In [2]:
BASE_URL = "http://web:8000/api/v0.0"
data_dir = os.getenv("DATA_DIR")

curl http://localhost:8000/api/v0.0/sources
curl http://localhost:8000/api/v0.0/sources?tdmq_id=e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6
curl http://localhost:8000/api/v0.0/sources?id="tdm/sensor_3"
curl http://localhost:8000/api/v0.0/sources?category=Station
curl http://localhost:8000/api/v0.0/sources?type=EnergyConsumptionMonitor
curl http://localhost:8000/api/v0.0/sources?controlledProperties=temperature
curl http://localhost:8000/api/v0.0/sources?controlledProperties=temperature,humidity
curl http://localhost:8000/api/v0.0/sources?after=2019-05-02T11:00:30Z
curl http://localhost:8000/api/v0.0/sources?stationary=false
curl http://localhost:8000/api/v0.0/sources?stationary=true
curl http://localhost:8000/api/v0.0/sources?brandName=Acme

## Query for constants

In [3]:
response = requests.get(f'{BASE_URL}/entity_types')
response.raise_for_status()
response.json()

[['Radar', 'MeteoRadarMosaic', None],
 ['Station', 'PointWeatherObserver', None],
 ['Station', 'TemperatureMosaic', None],
 ['Station', 'EnergyConsumptionMonitor', None]]

## Load sensor types

In [4]:
with open(os.path.join(data_dir, 'sources.json')) as f:
    data = json.load(f)['sources']

In [5]:
response = requests.post(f'{BASE_URL}/sources', json=data)
response.raise_for_status()
tdmq_ids = response.json()

In [6]:
tdmq_ids

['e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6',
 '2bb83248-27f4-53c9-b149-ca74c8c283ee',
 '374480f7-75c1-5747-a824-60676d7153a3',
 '42608b02-a82a-58ae-84fb-f358bf9b7494',
 '8b53505d-79b8-548d-b160-777d116cb6b2',
 '5df07546-b1d6-517f-8798-5abd79e7ebb1']

In [7]:
response = requests.get(f'{BASE_URL}/sources')
response.raise_for_status()
sources = response.json()

In [8]:
sources

[{'default_footprint': '{"type":"Point","coordinates":[9.22100000642642,30.0000000019687]}',
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_0',
  'tdmq_id': 'e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6'},
 {'default_footprint': '{"type":"Point","coordinates":[9.22200000642623,30.0030000019686]}',
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_1',
  'tdmq_id': '2bb83248-27f4-53c9-b149-ca74c8c283ee'},
 {'default_footprint': '{"type":"Point","coordinates":[9.20100000640809,29.6000000019797]}',
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_2',
  'tdmq_id': '374480f7-75c1-5747-a824-60676d7153a3'},
 {'default_footprint': '{"type":"Point","coordinates":[9.55100000643269,32.0000000018894]}',
  'entity_category': 'Station',
  'entity_type': 'EnergyConsumptionMonitor',
  'external_id': 'tdm/sensor_3',
  'tdmq_id': '42608b02-a82a-58ae-84fb-

In [10]:
for s in sources:
    response = requests.get(f'{BASE_URL}/sources/{s["tdmq_id"]}')
    response.raise_for_status()
    print('*'*20)
    print(response.json())                       

********************
{'default_footprint': '{"type":"Point","coordinates":[9.22100000642642,30.0000000019687]}', 'description': {'category': 'Station', 'controlledProperties': ['temperature'], 'default_footprint': {'coordinates': [9.221, 30.0], 'type': 'Point'}, 'description': {'alias': 'my desk', 'brandName': 'Acme', 'category': ['sensor'], 'function': ['sensing'], 'manufacturerName': 'Acme Inc.', 'modelName': 'Acme multisensor DHT11', 'type': 'TemperatureSensorDTH11'}, 'id': 'tdm/sensor_0', 'stationary': True, 'type': 'PointWeatherObserver'}, 'entity_category': 'Station', 'entity_type': 'PointWeatherObserver', 'external_id': 'tdm/sensor_0', 'stationary': True, 'tdmq_id': 'e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6'}
********************
{'default_footprint': '{"type":"Point","coordinates":[9.22200000642623,30.0030000019686]}', 'description': {'category': 'Station', 'controlledProperties': ['humidity'], 'default_footprint': {'coordinates': [9.222, 30.003], 'type': 'Point'}, 'description': {

In [11]:
for s in sources:
    response = requests.delete(f'{BASE_URL}/sources/{s["tdmq_id"]}')
    response.raise_for_status()

In [12]:
response = requests.get(f'{BASE_URL}/sources')
response.raise_for_status()
response.json()

[]